# Upper bound on the Hoffman constant for homogeneous inequality systems

Suppose $A\in \mathbb{R}^{m\times n}\setminus \{0\}$ and $P:=\{x:Ax\le 0\}$.

This notebook illustrates the procedure described in the paper "An easily computable upper bound on the Hoffman constant for homogeneous inequality systems" to compute an upper bound on the following *homogeneous Hoffman constant:*

$$
H_0(A) := \sup_{u\in \mathbb{R}^n \setminus P} \frac{\text{dist}(u,P)}{\text{dist}(Au, \mathbb{R}^m_-)}.$$  


In [1]:
import cvxpy as cp
import numpy as np
from boundHHoffman import boundH0

In [2]:
def genmatrix(m,n):
    # generate matrix with some non-trivial partition
    B = np.random.randint(m-3)+2
    s = np.random.randint(n-3)+2
    A = np.zeros((m,n))
    A[:B-1,:s] = np.random.randn(B-1,s)
    A[B-1,:s]=-A[:B-1,:s].T@np.random.rand(B-1)
    A[B:,:(n-1)] = np.random.randn(m-B,n-1)
    A[B:,n-1] = -np.random.rand(m-B) - A[B:,s:(n-1)]@np.random.randn(n-s-1)
    return A

def genLPmatrix(m,n):
    # generate instance based on LP duality
    A = np.random.randn(m,n)
    b = A@np.random.rand(n)-np.random.rand(m)
    c = A.T@np.random.randn(m)+np.random.rand(n)
    M = np.zeros((2*(m+n+1),m+n+1))
    M[0,1:(m+1)] = -b
    M[0,(m+1):] = c
    M[1:(m+1),0] = b
    M[m+1:m+n+1,0] = -c
    M[1:m+1,(m+1):] = -A
    M[m+1:m+n+1,1:(m+1)]=A.T
    M[m+n+1:,:] = np.identity(m+n+1)
    return M

def closest(u,A):
    # Find distance to and closest point to u in {Ax<=0}.  This is for reality check.
    m,n = A.shape
    y = cp.Variable(n)
    prob = cp.Problem(cp.Minimize(cp.quad_form(u-y,np.identity(n))),[A@y <= 0])
    prob.solve(solver = cp.GUROBI)
    return np.linalg.norm(u-y.value), y.value

Example using the above genmatrix function to generate an instance with non-trivial partition.

In [3]:
m=30; n=40
A = genmatrix(m,n)
h = boundH0(A)
h

Academic license - for non-commercial use only - expires 2023-10-16
Using license file /Users/javipena/gurobi.lic


1726.994709621617

Reality check

In [4]:
u=np.random.randn(n)
while max(A@u)<=0:
    u=np.random.randn(n)        
d,y = closest(u,A)
if (np.linalg.norm(y-u)/max(A@u)<=h):
    print('The upper bound checks out:',np.linalg.norm(y-u)/max(A@u),' <= ',h)
else:
    print('The upper bound does not check out',np.linalg.norm(y-u)/max(A@u),' > ',h)

The upper bound checks out: 0.4300218247964792  <=  1726.994709621617


Example using the above genLPmatrix function to generate an instance based on LP duality.  These are usually much more challenging.

In [5]:
A = genLPmatrix(40,40)
h = boundH0(A)
h

/Users/javipena/opt/anaconda3/lib/python3.8/site-packages/cvxpy/problems/problem.py:1278: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


7021472.237482714

In [6]:
m,n = A.shape
u=np.random.randn(n)
while max(A@u)<=0:
    u=np.random.randn(n)        
d,y = closest(u,A)
if (d/max(A@u)<=h):
    print('The upper bound checks out:',d/max(A@u),' <= ',h)
else:
    print('The upper bound does not check out',d/max(A@u),' > ',h)

The upper bound checks out: 0.3243943131604912  <=  7021472.237482714
